## Nayyera , Duaa , Mahum
## 21i1651 , 21i1667 , 21i1662

# Reading data and Preprocessing


In [1]:
import zipfile
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# prompt: read a zip folder from drive and unzip it

import zipfile
import os

# Specify the path to your zip file in Google Drive
zip_file_path = "/content/drive/MyDrive/Colab Notebooks/Deep Project.zip" # Replace with the actual path

# Specify the directory to extract the contents
extract_dir = "/content/extracted_files" # Or any desired directory

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_dir):
  os.makedirs(extract_dir)

try:
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
    print(f"Successfully unzipped {zip_file_path} to {extract_dir}")
except FileNotFoundError:
  print(f"Error: Zip file not found at {zip_file_path}")
except zipfile.BadZipFile:
  print(f"Error: Invalid zip file at {zip_file_path}")
except Exception as e:
  print(f"An unexpected error occurred: {e}")


Successfully unzipped /content/drive/MyDrive/Colab Notebooks/Deep Project.zip to /content/extracted_files


In [ ]:
import os
import cv2
import numpy as np

# Parameters
FRAME_SIZE = 64  # Resize frames to 64x64 for training
SELECTED_CLASSES = ['Biking', 'Diving', 'PullUps', 'Skiing', 'YoYo']  # Actions
def preprocess_and_save_videos(input_dir, output_dir, frame_size):
    """
    Preprocess videos by resizing frames to a specific size and saving them in the corresponding action folders.
    """
    for class_name in SELECTED_CLASSES:
        class_input_path = os.path.join(input_dir, class_name)
        class_output_path = os.path.join(output_dir, class_name)

        # Create output directory if it doesn't exist
        os.makedirs(class_output_path, exist_ok=True)

        for video_name in os.listdir(class_input_path):
            video_path = os.path.join(class_input_path, video_name)
            cap = cv2.VideoCapture(video_path)
            frames = []

            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                # Resize and convert frames
                frame = cv2.resize(frame, (frame_size, frame_size))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Grayscale
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)  # Convert back to 3-channel for saving
                frames.append(frame)

            cap.release()

            if frames:
                # Save processed video
                output_video_path = os.path.join(class_output_path, video_name)
                height, width = frames[0].shape[:2]
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                out = cv2.VideoWriter(output_video_path, fourcc, 30, (width, height))

                for frame in frames:
                    out.write(frame)
                out.release()
                print(f"Processed and saved: {output_video_path}")


In [ ]:
# Paths for input and output
input_dataset_path = r'c:\Users\hp\Desktop\NAYYERA\nbayyera haha\train'  # Update to your dataset path
output_dataset_path = r'processed_videos'

# Preprocess and save videos
preprocess_and_save_videos(input_dataset_path, output_dataset_path, FRAME_SIZE)

Processed and saved: processed_videos\Biking\v_Biking_g01_c01.avi
Processed and saved: processed_videos\Biking\v_Biking_g01_c02.avi
Processed and saved: processed_videos\Biking\v_Biking_g01_c03.avi
Processed and saved: processed_videos\Biking\v_Biking_g01_c04.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c02.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c03.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c04.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c06.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c07.avi
Processed and saved: processed_videos\Biking\v_Biking_g03_c01.avi
Processed and saved: processed_videos\Biking\v_Biking_g03_c02.avi
Processed and saved: processed_videos\Biking\v_Biking_g03_c03.avi
Processed and saved: processed_videos\Biking\v_Biking_g03_c04.avi
Processed and saved: processed_videos\Biking\v_Biking_g04_c01.avi
Processed and saved: processed_videos\Biking\v_Biking_g04_c02.avi
Processed 

# SEQUENCE GENERATING

In [ ]:
import os
import numpy as np
import cv2

# Parameters
SEQUENCE_LENGTH = 10  # Input sequence length
PREDICT_FRAMES = 5  # Frames to predict

def load_video_frames(video_path, frame_size):
    """Load video frames from a file and preprocess."""
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (frame_size, frame_size))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Grayscale
        frames.append(frame)

    cap.release()
    return np.array(frames)  # Shape: (num_frames, frame_size, frame_size)

In [ ]:
def save_sequences(sequence_folder, action, video_name, X, y):
    """Save the input-output sequences as .npy files."""
    action_folder = os.path.join(sequence_folder, action)
    os.makedirs(action_folder, exist_ok=True)  # Create action folder if not exists

    video_base = os.path.splitext(video_name)[0]
    X_file = os.path.join(action_folder, f"{video_base}_X.npy")
    y_file = os.path.join(action_folder, f"{video_base}_y.npy")

    np.save(X_file, X)
    np.save(y_file, y)

In [ ]:
def generate_sequences(dataset_path, sequence_folder, sequence_length, predict_frames, frame_size):
    """Generate input-output sequences for ConvLSTM training and save them."""
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue  # Skip if not a directory

        for video_name in os.listdir(class_path):
            video_path = os.path.join(class_path, video_name)
            frames = load_video_frames(video_path, frame_size)

            # Ensure enough frames for at least one sequence
            required_frames = sequence_length + predict_frames
            if len(frames) < required_frames:
                print(f"Skipping {video_name} (Insufficient frames)")
                continue

            # Generate sequences
            num_sequences = len(frames) - required_frames + 1
            X, y = [], []
            for i in range(num_sequences):
                X.append(frames[i:i + sequence_length])  # Input sequence
                y.append(frames[i + sequence_length:i + sequence_length + predict_frames])  # Target sequence

            X = np.array(X)  # Shape: (num_samples, sequence_length, frame_size, frame_size)
            y = np.array(y)  # Shape: (num_samples, predict_frames, frame_size, frame_size)

            # Save sequences
            save_sequences(sequence_folder, class_name, video_name, X, y)


In [ ]:
# Paths
dataset_path = r'processed_videos'
sequence_folder = r'sequences'
frame_size = 64

# Generate and save sequences
generate_sequences(dataset_path, sequence_folder, SEQUENCE_LENGTH, PREDICT_FRAMES, frame_size)
print("Sequences generated and saved.")


Sequences generated and saved.


# PRED RNN FROM SCRATCH

In [ ]:
!pip install tensorflow

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

class FramePredictionDataset(Dataset):
    def __init__(self, sequence_dir, sequence_length=10, predict_frames=5):
        self.inputs = []
        self.targets = []

        # Detailed logging of data loading
        print("=" * 50)
        print("DATA LOADING DIAGNOSTIC INFORMATION")
        print("=" * 50)

        # Scan through directory structure
        for class_dir in os.listdir(sequence_dir):
            class_path = os.path.join(sequence_dir, class_dir)

            # Ensure it's a directory
            if not os.path.isdir(class_path):
                continue

            print(f"\nProcessing class directory: {class_dir}")

            # Process each video's sequences
            for file in os.listdir(class_path):
                if file.endswith('_X.npy'):
                    # Full file paths
                    input_file_path = os.path.join(class_path, file)
                    target_file_path = os.path.join(class_path, file.replace('_X.npy', '_y.npy'))

                    # Load sequences
                    input_sequences = np.load(input_file_path)
                    target_sequences = np.load(target_file_path)

                    # Detailed input sequence diagnostics
                    print("\nInput Sequence Diagnostics:")
                    print(f"Input file: {file}")
                    print(f"Input sequences shape: {input_sequences.shape}")
                    print(f"Input sequences dtype: {input_sequences.dtype}")
                    print(f"Input sequences min/max: {input_sequences.min()}, {input_sequences.max()}")

                    # Detailed target sequence diagnostics
                    print("\nTarget Sequence Diagnostics:")
                    print(f"Target file: {file.replace('_X.npy', '_y.npy')}")
                    print(f"Target sequences shape: {target_sequences.shape}")
                    print(f"Target sequences dtype: {target_sequences.dtype}")
                    print(f"Target sequences min/max: {target_sequences.min()}, {target_sequences.max()}")

                    # Additional shape analysis
                    print("\nShape Analysis:")
                    if input_sequences.ndim == 3:
                        print("Input sequences are 3D (no channel dimension)")
                    elif input_sequences.ndim == 4:
                        print("Input sequences are 4D (with channel dimension)")
                        print(f"Channel dimension is at axis: {input_sequences.ndim - 1}")

                    if target_sequences.ndim == 3:
                        print("Target sequences are 3D (no channel dimension)")
                    elif target_sequences.ndim == 4:
                        print("Target sequences are 4D (with channel dimension)")
                        print(f"Channel dimension is at axis: {target_sequences.ndim - 1}")

                    # Collect sequences
                    self.inputs.extend(list(input_sequences))
                    self.targets.extend(list(target_sequences))

        # Convert to numpy arrays
        self.inputs = np.array(self.inputs)
        self.targets = np.array(self.targets)

        # Final dataset summary
        print("\n" + "=" * 50)
        print("FINAL DATASET SUMMARY")
        print("=" * 50)
        print(f"Total input sequences: {len(self.inputs)}")
        print(f"Total target sequences: {len(self.targets)}")
        print(f"Input sequences shape: {self.inputs.shape}")
        print(f"Target sequences shape: {self.targets.shape}")

        # Verify sequence lengths
        if len(self.inputs) > 0:
            print("\nFirst input sequence details:")
            print(f"Shape: {self.inputs[0].shape}")
            print(f"Data type: {self.inputs[0].dtype}")
            print(f"Min/Max values: {self.inputs[0].min()}, {self.inputs[0].max()}")

        if len(self.targets) > 0:
            print("\nFirst target sequence details:")
            print(f"Shape: {self.targets[0].shape}")
            print(f"Data type: {self.targets[0].dtype}")
            print(f"Min/Max values: {self.targets[0].min()}, {self.targets[0].max()}")

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        # Convert to torch tensors with additional checks
        input_tensor = torch.tensor(self.inputs[idx], dtype=torch.float32)
        target_tensor = torch.tensor(self.targets[idx], dtype=torch.float32)

        # Diagnostic print for each item
        print("\nTensor Conversion Diagnostics:")
        print(f"Input tensor shape: {input_tensor.shape}")
        print(f"Target tensor shape: {target_tensor.shape}")

        # Optional: Add dimension handling if needed
        if input_tensor.dim() == 3:
            input_tensor = input_tensor.unsqueeze(0)  # Add channel dimension if missing
        if target_tensor.dim() == 3:
            target_tensor = target_tensor.unsqueeze(0)  # Add channel dimension if missing

        return input_tensor, target_tensor

def main():
    # Configurations
    sequence_dir = '/content/extracted_files/Nayyera Deep Project/sequences'

    # Create dataset with detailed diagnostics
    dataset = FramePredictionDataset(sequence_dir)

    # Create dataloader with verbose settings
    dataloader = DataLoader(
        dataset,
        batch_size=16,  # Adjust as needed
        shuffle=True,
        num_workers=2
    )

    # Iterate through first few batches to verify
    print("\n" + "=" * 50)
    print("DATALOADER BATCH VERIFICATION")
    print("=" * 50)
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        print(f"\nBatch {batch_idx + 1}:")
        print(f"Inputs shape: {inputs.shape}")
        print(f"Targets shape: {targets.shape}")

        # Only print first few batches
        if batch_idx >= 2:
            break

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
Target file: v_YoYo_g05_c01_y.npy
Target sequences shape: (178, 5, 64, 64)
Target sequences dtype: uint8
Target sequences min/max: 0, 236

Shape Analysis:
Input sequences are 4D (with channel dimension)
Channel dimension is at axis: 3
Target sequences are 4D (with channel dimension)
Channel dimension is at axis: 3

Input Sequence Diagnostics:
Input file: v_YoYo_g06_c02_X.npy
Input sequences shape: (170, 10, 64, 64)
Input sequences dtype: uint8
Input sequences min/max: 0, 255

Target Sequence Diagnostics:
Target file: v_YoYo_g06_c02_y.npy
Target sequences shape: (170, 5, 64, 64)
Target sequences dtype: uint8
Target sequences min/max: 0, 255

Shape Analysis:
Input sequences are 4D (with channel dimension)
Channel dimension is at axis: 3
Target sequences are 4D (with channel dimension)
Channel dimension is at axis: 3

Input Sequence Diagnostics:
Input file: v_YoYo_g21_c01_X.npy
Input sequences shape: (171, 10, 64, 64)
Input sequences dtyp

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import traceback
import psutil
import gc

def check_memory():
    process = psutil.Process(os.getpid())
    print(f"Memory Usage: {process.memory_info().rss / 1024 / 1024:.2f} MB")

def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()

class ZoneoutCell(nn.Module):  # to save prev information
    def __init__(self, cell, zoneout_prob=0.1):
        super(ZoneoutCell, self).__init__()
        self.cell = cell
        self.zoneout_prob = zoneout_prob

    def forward(self, x, h, c):
        h_next, c_next = self.cell(x, h, c)

        if self.training:
            h_mask = torch.bernoulli(torch.ones_like(h) * self.zoneout_prob)
            c_mask = torch.bernoulli(torch.ones_like(c) * self.zoneout_prob)

            h_next = h * h_mask + (1 - h_mask) * h_next
            c_next = c * c_mask + (1 - c_mask) * c_next

        return h_next, c_next

class PredRNNCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size=3, bias=True):
        super(PredRNNCell, self).__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2

        # Spatial-Temporal Memory Module
        self.spatiotemporal_memory = nn.Conv2d(
            hidden_channels, hidden_channels,
            kernel_size=kernel_size,
            padding=self.padding,
            bias=bias
        )

        # Main Gates
        self.conv_gates = nn.Conv2d(
            in_channels=input_channels + hidden_channels,
            out_channels=4 * hidden_channels,
            kernel_size=kernel_size,
            padding=self.padding,
            bias=bias
        )

    def forward(self, x, h, c, m):
        # Combine inputs
        combined = torch.cat([x, h], dim=1)

        # Calculate gates
        gates = self.conv_gates(combined)
        i, f, o, g = torch.split(gates, self.hidden_channels, dim=1)

        # Apply activations
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        # Memory update with spatial-temporal memory
        m_next = self.spatiotemporal_memory(m)

        # Cell state and hidden state updates
        c_next = f * c + i * g
        m_next = m_next + c_next
        h_next = o * torch.tanh(m_next)

        return h_next, c_next, m_next

class PredRNNModel(nn.Module):
    def __init__(self, input_channels=1, hidden_channels=64,
                 num_layers=2, predict_frames=5):
        super(PredRNNModel, self).__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers
        self.predict_frames = predict_frames

        # Encoder
        self.encoder = nn.Conv2d(
            in_channels=input_channels,
            out_channels=hidden_channels,
            kernel_size=3,
            padding=1
        )

        # PredRNN layers with Zoneout
        self.layers = nn.ModuleList([
            ZoneoutCell(PredRNNCell(
                input_channels=hidden_channels if i == 0 else hidden_channels,
                hidden_channels=hidden_channels
            )) for i in range(num_layers)
        ])

        # Decoder
        self.decoder = nn.Conv2d(
            in_channels=hidden_channels,
            out_channels=input_channels,
            kernel_size=3,
            padding=1
        )

    def forward(self, x):
        batch_size, seq_len, height, width = x.shape
        x = x.unsqueeze(1)
        device = x.device

        # Encoder
        encoded_frames = [self.encoder(x[:, :, t, :, :]) for t in range(seq_len)]

        # Initialize hidden states, cell states, and memory states
        hidden_states = [
            torch.zeros(batch_size, self.hidden_channels, height, width, device=device)
            for _ in range(self.num_layers)
        ]
        cell_states = [
            torch.zeros(batch_size, self.hidden_channels, height, width, device=device)
            for _ in range(self.num_layers)
        ]
        memory_states = [
            torch.zeros(batch_size, self.hidden_channels, height, width, device=device)
            for _ in range(self.num_layers)
        ]

        # Process input sequence
        for t in range(seq_len):
            frame = encoded_frames[t]
            for i in range(self.num_layers):
                h, c = hidden_states[i], cell_states[i]
                m = memory_states[i]
                h, c, m = self.layers[i].cell(frame, h, c, m)
                hidden_states[i], cell_states[i], memory_states[i] = h, c, m
                frame = h

        # Generate future predictions
        predicted_frames = []
        current_frame = encoded_frames[-1]

        for _ in range(self.predict_frames):
            for i in range(self.num_layers):
                h, c = hidden_states[i], cell_states[i]
                m = memory_states[i]
                h, c, m = self.layers[i].cell(current_frame, h, c, m)
                hidden_states[i], cell_states[i], memory_states[i] = h, c, m
                current_frame = h

            # Decode frame
            pred_frame = self.decoder(current_frame)
            predicted_frames.append(pred_frame.squeeze(1))

        return torch.stack(predicted_frames, dim=1)
class FramePredictionDataset(Dataset):
    def __init__(self, sequence_dir, sequence_length=10, predict_frames=5, max_sequences=None):
        self.inputs = []
        self.targets = []

        # Track loading progress and memory
        total_loaded_sequences = 0

        # Load sequences from directory
        for class_dir in os.listdir(sequence_dir):
            class_path = os.path.join(sequence_dir, class_dir)

            # Process each video's sequences
            for file in os.listdir(class_path):
                if file.endswith('_X.npy'):
                    try:
                        # Load input and target sequences
                        input_sequences = np.load(os.path.join(class_path, file))
                        target_sequences = np.load(os.path.join(class_path, file.replace('_X.npy', '_y.npy')))

                        # Normalize sequences
                        input_sequences = input_sequences.astype(np.float32) / 255.0
                        target_sequences = target_sequences.astype(np.float32) / 255.0

                        self.inputs.extend(input_sequences)
                        self.targets.extend(target_sequences)

                        total_loaded_sequences += len(input_sequences)

                        # Optional: Limit total sequences to prevent memory issues
                        if max_sequences and total_loaded_sequences >= max_sequences:
                            break

                    except Exception as e:
                        print(f"Error loading file {file}: {e}")
                        traceback.print_exc()
                        continue

        # Convert to numpy arrays
        self.inputs = np.array(self.inputs)
        self.targets = np.array(self.targets)

        print("Input shape:", self.inputs.shape)
        print("Target shape:", self.targets.shape)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.inputs[idx], dtype=torch.float32),
            torch.tensor(self.targets[idx], dtype=torch.float32)
        )

def train_model(model, dataloader, num_epochs=10, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training on: {device}")

    model.to(device)
    model.train()

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    losses = []
    try:
        for epoch in range(num_epochs):
            epoch_loss = 0.0
            batch_count = 0

            for inputs, targets in dataloader:
                try:
                    # Memory and performance checks
                    check_memory()

                    inputs, targets = inputs.to(device), targets.to(device)

                    optimizer.zero_grad()

                    # Predictions
                    predictions = model(inputs)

                    # Compute loss
                    loss = criterion(predictions, targets)

                    loss.backward()
                    optimizer.step()

                    epoch_loss += loss.item()
                    batch_count += 1

                    # Clear memory after each batch
                    clear_memory()

                except RuntimeError as e:
                    print(f"Error processing batch: {e}")
                    traceback.print_exc()
                    clear_memory()
                    continue

            avg_loss = epoch_loss / max(batch_count, 1)
            losses.append(avg_loss)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

            # Plot and save intermediate results
            if (epoch + 1) % 5 == 0:
                plt.figure(figsize=(10, 5))
                plt.plot(losses)
                plt.title('Training Loss')
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.savefig(f'training_loss_epoch_{epoch+1}.png')
                plt.close()

    except Exception as e:
        print(f"Training interrupted: {e}")
        traceback.print_exc()

    finally:
        # Save final model
        torch.save(model.state_dict(), 'video_prediction_model.pth')
        print("Model saved.")

def main():
    # Configurations
    sequence_dir = '/content/extracted_files/Nayyera Deep Project/sequences'
    sequence_length = 10
    predict_frames = 5

    # Adjust batch size and limit sequences to prevent memory issues
    batch_size = 8  # Reduced from 16
    num_epochs = 30

    try:
        # Create dataset and dataloader
        dataset = FramePredictionDataset(
            sequence_dir,
            sequence_length=sequence_length,
            predict_frames=predict_frames,
            max_sequences=1000  # Limit total sequences
        )

        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,  # Set to 0 if multiprocessing causes issues
            pin_memory=True
        )

        # Initialize model
        model = PredRNNModel(
            input_channels=1,
            hidden_channels=64,
            num_layers=2,
            predict_frames=predict_frames
        )

        # Train model
        train_model(model, dataloader, num_epochs=num_epochs)

    except Exception as e:
        print(f"Main process error: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB
Memory Usage: 1307.38 MB

In [5]:
import os
import torch
import numpy as np
import matplotlib.pyplot as pltxc
import matplotlib.animation as animation
import torch.nn as nn
import torch.nn.functional as F
from skimage.metrics import structural_similarity as compare_ssim


# (Model and other utility functions remain the same)


def evaluate_predictions(predictions, targets):
    """
    Evaluate frame prediction accuracy using MSE and SSIM.

    Args:
        predictions (torch.Tensor): Predicted frames, shape (batch_size, predict_frames, height, width).
        targets (torch.Tensor): Ground truth frames, shape (batch_size, predict_frames, height, width).

    Returns:
        dict: Mean MSE and SSIM scores across all predicted frames.
    """
    predictions = predictions.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()

    mse_list = []
    ssim_list = []

    for batch_idx in range(predictions.shape[0]):
        for frame_idx in range(predictions.shape[1]):
            pred_frame = predictions[batch_idx, frame_idx]
            target_frame = targets[batch_idx, frame_idx]

            # Compute MSE
            mse = np.mean((pred_frame - target_frame) ** 2)
            mse_list.append(mse)

            # Compute SSIM with data_range
            ssim = compare_ssim(pred_frame, target_frame, data_range=1.0)
            ssim_list.append(ssim)

    avg_mse = np.mean(mse_list)
    avg_ssim = np.mean(ssim_list)

    return {"MSE": avg_mse, "SSIM": avg_ssim}


def create_sequence_video(input_frames, target_frames, predicted_frames, output_filename='sequence_comparison.mp4'):
    """
    Create a video comparing input, ground truth, and predicted frames.
    """
    # Prepare figure and axis
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle('Frame Sequence Comparison', fontsize=16)

    # Initialize images
    input_img = ax1.imshow(input_frames[0], cmap='gray')
    target_img = ax2.imshow(target_frames[0], cmap='gray')
    pred_img = ax3.imshow(predicted_frames[0], cmap='gray')

    # Set titles
    ax1.set_title('Input Frames')
    ax2.set_title('Ground Truth Frames')
    ax3.set_title('Predicted Frames')

    # Remove axes
    for ax in [ax1, ax2, ax3]:
        ax.axis('off')

    # Combine all frames for iteration
    all_frames = list(zip(input_frames, target_frames, predicted_frames))

    # Animation update function
    def update(frame):
        input_frame, target_frame, pred_frame = frame
        input_img.set_array(input_frame)
        target_img.set_array(target_frame)
        pred_img.set_array(pred_frame)
        return input_img, target_img, pred_img

    # Create animation
    anim = animation.FuncAnimation(
        fig,
        update,
        frames=all_frames,
        interval=500,  # 500 ms between frames
        blit=True
    )

    # Save video
    anim.save(output_filename, writer='ffmpeg', fps=2)
    print(f"Video saved as {output_filename}")
    plt.close(fig)


def visualize_and_create_video(model_path, sequence_path, sequence_length=10, predict_frames=5):
    """
    Visualize and create video from model predictions, evaluate using MSE and SSIM.
    """
    # Set up device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize model
    model = PredRNNModel(
        input_channels=1,
        hidden_channels=64,
        num_layers=2,
        predict_frames=predict_frames
    )

    # Load model weights
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

    # Process sequences
    for class_dir in os.listdir(sequence_path):
        class_path = os.path.join(sequence_path, class_dir)

        for file in os.listdir(class_path):
            if file.endswith('_X.npy'):
                # Load input sequence
                input_sequences = np.load(os.path.join(class_path, file))
                target_sequences = np.load(os.path.join(class_path, file.replace('_X.npy', '_y.npy')))

                # Normalize and convert to tensor
                input_sequences = input_sequences.astype(np.float32) / 255.0
                target_sequences = target_sequences.astype(np.float32) / 255.0

                # Convert to tensor and add batch dimension
                input_tensor = torch.tensor(input_sequences[0]).unsqueeze(0).to(device)
                target_tensor = torch.tensor(target_sequences[0]).unsqueeze(0).to(device)

                # Generate predictions
                with torch.no_grad():
                    predictions = model(input_tensor)

                # Evaluate predictions
                metrics = evaluate_predictions(predictions, target_tensor)
                print(f"Mean MSE: {metrics['MSE']:.4f}, Mean SSIM: {metrics['SSIM']:.4f}")

                # Convert to numpy
                input_np = input_tensor.cpu().numpy()[0]
                target_np = target_sequences[0]
                pred_np = predictions.cpu().numpy()[0]

                # Create video
                create_sequence_video(
                    input_np,
                    target_np,
                    pred_np,
                    output_filename='sequence_comparison.mp4'
                )

                return  # Visualize first sequence found


# Example usage
if __name__ == "__main__":
    model_path = '/content/video_prediction_model.pth'  # Path to your saved model
    sequence_dir = '/content/extracted_files/Nayyera Deep Project/sequences'  # Path to sequences directory

    visualize_and_create_video(
        model_path=model_path,
        sequence_path=sequence_dir
    )


<ipython-input-5-13e5ca6fa7e5>:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Mean MSE: 0.0012, Mean SSIM: 0.9422
Video saved as sequence_comparison.mp4
